In [21]:
import requests as re
import pandas as pd
import json
import string
table = str.maketrans('', '', string.punctuation)

f = open("myapikey.txt", "r")
apikey= f.read() #Insert your own Google Places API Key here if you'd like to test the notebook
f.close() 

from IPython.display import display 
from IPython.display import clear_output

def makeDFList(tempdf):
    global firstLoc
    try: #Try Making Distance Column to Position 0 for Reference
        tempdf['distance']=tempdf['Lat']+tempdf['Lng']- firstLoc.iloc[0]['Lat'] - firstLoc.iloc[0]['Lng'] #Put Real Dis Calc Here
    except:
        pass
    
    display(tempdf)
    entry= input('Edit or remove row from list? Type numbers (space seperator), or type exit to stop').lower()
    while entry != 'exit':
        try:
            entry= [int(x) for x in entry.split()]
            tempdf.drop(index=entry, inplace=True)
            clear_output()
            display(tempdf)
        except:
            input('Error with input or dropping rows, press enter to continue')    
        entry= input('Continue to remove rows? Type exit to stop.').lower()
    
    tempdf.drop(columns='distance',inplace=True, errors='ignore')

        
def searchTerms(query=None, first=False):
    if query==None:
        query= input('Search Terms?')
        
    query=(query
           .translate(table)
           .lower()
           .replace(' ','+')
          )
    url=('https://maps.googleapis.com/maps/api/place/textsearch/json?query={}&key={}'
         .format(query,apikey)
        )
    print('Searched for',query)
    res = re.get(url)
    tempdf=pd.json_normalize(res.json()['results'])[['name','formatted_address','geometry.location.lat','geometry.location.lng','place_id']]
    tempdf.columns=['Name','Address','Lat','Lng','place_id']

    makeDFList(tempdf)

    if first:
        if len(tempdf)>1:
            print('Length Error, taking first element as home')
        return tempdf.iloc[0:1] #Return only 1 element
    else:
        return tempdf #return searched and edited DF


def setStartLocation(query=None):
    global firstLoc
    firstLoc=searchTerms(query, first=True)
    return firstLoc

def addToDF():
    global firstLoc
    firstLoc= pd.concat([firstLoc, searchTerms()],ignore_index=True)


In [22]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = disMatrix 
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    #print('Objective: {} seconds'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = ''#Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {}'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}'.format(manager.IndexToNode(index))
    #print(plan_output)
    #plan_output += 'Route distance: {}seconds\n'.format(route_distance)
    #print(plan_output)
    #print(plan_output)
    #print(type(plan_output))
    return plan_output


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
#     if solution:
#         print_solution(manager, routing, solution)
    return solution.ObjectiveValue(), print_solution(manager, routing, solution)

In [23]:
print('Set Start Location')
setStartLocation()
choice=None
while choice!='3':
    choice=input('Pick 1 for add to List, 2 to Edit List, or 3 to run TSP')
    clear_output()
    try:
        assert choice in ['1','2','3']
        if choice=='1':
            print('Add to List')
            addToDF()
        if choice=='2':
            print('Edit List')
            makeDFList(firstLoc)
    except:
        print('Try Again')

if len(firstLoc)>10:
    print('List Too Long and Truncating to first 10')
testdf=firstLoc.iloc[0: min(10,len(firstLoc))]
testdf['place_id']
placeIDJoined='place_id:'+'|place_id:'.join(testdf['place_id'])
placeIDJoined
url=('https://maps.googleapis.com/maps/api/distancematrix/json?origins={}&destinations={}&departure_time=now&key={}'
     .format(placeIDJoined,placeIDJoined,apikey)
    )
res = re.get(url)

disMatrix=[]
for i in range(len(res.json()['rows'])):
    disMatrix.append(list(pd.json_normalize(res.json()['rows'][i]['elements'])['duration_in_traffic.value']))

time, strList=main()

strToNumList= [int(x) for x in strList.split()]

orderedDF= testdf.iloc[(strToNumList)]

orderedlink=orderedDF['Lat'].astype('string')+','+orderedDF['Lng'].astype('string')

print('{} mins for this trip. Please visit in this order for the fastest trip'.format(time//60))
display(orderedDF[['Name','Address']].reset_index().drop(columns='index'))
mapurl='https://www.google.com/maps/dir/'+'/'.join(orderedlink)
mapurl

List Too Long and Truncating to first 10
53 mins for this trip. Please visit in this order for the fastest trip


,Name,Address
0,"Scarborough, ON M1H 3J5","Scarborough, ON M1H 3J5, Canada"
1,Shoppers Drug Mart,"300 Borough Dr, Scarborough, ON M1P 4P5, Canada"
2,Beauty Boutique by Shoppers Drug Mart,"300 Borough Dr, Scarborough, ON M1P 4P5, Canada"
3,Shoppers Drug Mart,"1235 McCowan Rd, Scarborough, ON M1H 3K3, Canada"
4,Shoppers Drug Mart,"685 McCowan Rd, Scarborough, ON M1J 1K2, Canada"
5,Shoppers Drug Mart,"629 Markham Rd, Scarborough, ON M1H 2A4, Canada"
6,Shoppers Drug Mart,"Centenary Health Centre, 2863 Ellesmere Rd #20..."
7,Shoppers Drug Mart,"1400 Neilson Rd, Scarborough, ON M1B 0C2, Canada"
8,Shoppers Drug Mart,"1571 Sandhurst Cir #206, Scarborough, ON M1V 1..."
9,Shoppers Drug Mart,"1780 Markham Rd A, Scarborough, ON M1B 2W2, Ca..."


'https://www.google.com/maps/dir/43.7822142,-79.2447625/43.776874,-79.256654/43.776129,-79.257601/43.773305,-79.251662/43.75447399999999,-79.24488099999999/43.759825,-79.225268/43.780599,-79.20500899999999/43.80961,-79.222729/43.808894,-79.269854/43.794384,-79.240118/43.7822142,-79.2447625'